In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Original and output data folders
original = Path("../original/")
output = Path("../output/")

# Management area

In [3]:
# Information on farms
ManagementArea = pd.read_excel(original / "ManagementArea.xlsx")
ManagementArea.head()

,Id,FarmRegistrationId,CompanyName,AccountManagerFirstName,AccountManagerLastName,ContactNumber,ContactEmail,MilkPrice,VetContactNumber,VetContactEmail,IsActive
0,1566,ES140030000379,Enrique Galán,Enrique,Galán Hinojosa,629590970,enriquegalanh@hotmail.com,1.35,6.578609e+08,alvaro@boalvet.ai,True
1,695,1990,Hermanos Matunez,Eleuterio,Sanchez,6999996696,this.is@test.com,0.10,6.999670e+08,this.is@test.com,True
2,1903,ES290110000451,Hoyo del Conejo,Jose,Fernández Sánchez,658209914,pepecasabermeja94@gmail.com,1.00,6.156151e+08,adsgmontesdemalaga@gmail.com,True
3,1906,ES110010000215,Jaime Gonzalez,Ildefonso,Gonzalez Marquez,670609994,ilde_15_alcala@hotmail.com,1.00,6.743589e+08,raul@boalvet.ai,True
4,1867,ES110050000063,Lopicomo SL Ovejas,José Luis,Holgado Carrero,678415381,info@quesospajarete.com,0.10,9.115309e+10,m.costella@substrate.ai,True


In [4]:
# Rename Id to ManagementAreaId
ManagementArea.rename(columns={"Id": "ManagementAreaId"}, inplace=True)

# Milk measurements

In [5]:
# Milk characteristics
MilkMeasurements = pd.read_excel(original / "MilkMeasurements.xlsx")
MilkMeasurements.head()

,Id,ManagementAreaId,SampleDate,NumberOfAnimals,Fat,Protein,Lactose,Casein,ES,EQ,...,SampleId,MilkQualityValue,LastModified,IsConsolidated,Temperature,Stability,AflatoxinsN,Aflatoxins,TemperatureMin,TemperatureMax
0,1484,1566,2022-05-16,0,5.80,5.34,0.00,0.0,11.15,11.14,...,O/41/0005703,-1,2022-05-17 18:00:15.840,True,NaN,NaN,NaN,NaN,NaN,NaN
1,1485,750,2022-05-14,0,4.07,3.68,4.56,0.0,7.75,7.75,...,694003043487,-1,2022-05-17 18:00:07.190,True,NaN,NaN,NaN,NaN,NaN,NaN
2,1486,750,2022-05-12,0,4.16,3.77,4.57,0.0,7.93,7.93,...,694002447125,-1,2022-05-17 18:00:07.190,True,NaN,NaN,NaN,NaN,NaN,NaN
3,1487,1524,2022-05-14,0,3.63,3.39,4.54,0.0,7.02,7.02,...,694004478578,-1,2022-05-17 18:00:10.837,True,NaN,NaN,NaN,NaN,NaN,NaN
4,1488,1524,2022-05-12,0,4.69,3.34,4.42,0.0,8.03,8.03,...,694004478424,-1,2022-05-17 18:00:10.837,True,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# How many values at 0 (a.k.a null) per column?
(MilkMeasurements == 0).sum()

Id                     0
ManagementAreaId       0
SampleDate             0
NumberOfAnimals     3529
Fat                  137
Protein              137
Lactose             1473
Casein              2977
ES                   137
EQ                   137
Bacteria             532
SomaticCellCount     316
Urea                 893
FreezingPoint        141
Inhibitors          3512
SampleId               0
MilkQualityValue     302
LastModified           0
IsConsolidated       107
Temperature            0
Stability              0
AflatoxinsN            0
Aflatoxins             0
TemperatureMin         0
TemperatureMax         0
dtype: int64

In [7]:
# Drop unusable, unecessary and null columns
cols_to_drop = [
    "NumberOfAnimals",
    "Lactose",
    "Casein",
    "Inhibitors",
    "SampleId",
    "MilkQualityValue",
    "LastModified",
    "IsConsolidated",
    "Temperature",
    "Stability",
    "AflatoxinsN",
    "Aflatoxins",
    "TemperatureMin",
    "TemperatureMax",
]
MilkMeasurements.drop(columns=cols_to_drop, inplace=True)

In [8]:
# Correct formatting errors on freezing point
def reformat_FreezingPoint(FreezingPoint):
    if FreezingPoint < 0:
        FreezingPoint = -FreezingPoint
    if FreezingPoint > 1:
        FreezingPoint /= 1000

    return FreezingPoint


MilkMeasurements["FreezingPoint"] = MilkMeasurements["FreezingPoint"].apply(
    reformat_FreezingPoint
)

In [9]:
# Merge with farms
MilkMeasurements = MilkMeasurements.merge(
    ManagementArea[["ManagementAreaId"]], on="ManagementAreaId"
)

# Retrieve animal breeds and species and add in milk measurements

In [10]:
# Breed
AnimalBreed = pd.read_excel(original / "AnimalBreed.xlsx")
AnimalBreed.rename(columns={"Id": "BreedId", "Name": "Breed"}, inplace=True)

# Species
AnimalSpecies = pd.read_excel(original / "AnimalSpecies.xlsx")
AnimalSpecies.rename(columns={"Id": "SpeciesId", "Name": "Species"}, inplace=True)

# Breed and species by farms
AnimalGroup = pd.read_excel(original / "AnimalGroup.xlsx")
AnimalGroup = AnimalGroup[["ManagementAreaId", "BreedId", "SpeciesId"]]

# Take ids to labels
AnimalBreedAndSpecies = pd.merge(AnimalBreed, AnimalSpecies, on="SpeciesId")
Animals = pd.merge(AnimalGroup, AnimalBreedAndSpecies, on=["BreedId", "SpeciesId"])
Animals.drop(columns=["BreedId", "SpeciesId"], inplace=True)

# Map Spanish species names to English ones 
species_map = {
    "Cabra": "Goat",
    "Oveja": "Sheep",
    "Vaca": "Cow",
}

Animals["Species"] = Animals["Species"].map(species_map)

# Overview
Animals.head()

,ManagementAreaId,Breed,Species
0,1845,Lacaune,Sheep
1,1915,Lacaune,Sheep
2,1914,Lacaune,Sheep
3,1893,Payoya,Goat
4,1892,Payoya,Goat


In [11]:
# Merge breed and species labels to milk measurements
MilkMeasurements = pd.merge(MilkMeasurements, Animals, on="ManagementAreaId")

# Melt milk measurements and merge with measurement types

In [12]:
MeasurementType = pd.read_excel(original / "MeasurementType.xlsx")
MeasurementType

,Id,Name
0,1,Cantidad de leche
1,2,Extracto Quesero
2,3,Células Somáticas
3,4,Grasa
4,5,Proteína
5,6,Bacterias
6,7,Urea
7,8,Extracto Seco
8,9,Caseína
9,10,Lactosa


In [13]:
# Map Spanish species names to English ones 
species_map = {
    "Cabra": "Goat",
    "Oveja": "Sheep",
    "Vaca": "Cow",
}

Animals["Species"]= Animals["Species"].map(species_map)

In [14]:
# Milk characteristics labels in Spanish
MeasurementType = pd.read_excel(original / "MeasurementType.xlsx")

# Map Spanish species names to English ones to match MilkMeasurements column names
measurements_map = {
    "Cantidad de leche": "Quantity",
    "Extracto Quesero": "EQ",
    "Células Somáticas": "SomaticCellCount",
    "Grasa": "Fat",
    "Proteína": "Protein",
    "Bacterias": "Bacteria",
    "Urea": "Urea",
    "Extracto Seco": "ES",
    "Caseína": "Casein",
    "Lactosa": "Lactose",
    "Punto Crioscópico": "FreezingPoint",
    "Número de animales": "NumberOfAnimals",
}

MeasurementType["Measurement"] = MeasurementType["Name"].map(measurements_map)

# Drop Spanish measurement names
MeasurementType.drop(columns="Name", inplace=True)

# Drop quantity and number of animals because their values are unavailable, casein and lactose because they're unused in the recommandation algorithm
MeasurementType = MeasurementType[
    ~MeasurementType["Measurement"].isin(
        ["Quantity", "NumberOfAnimals", "Casein", "Lactose"]
    )
]

# Rename Id to MeasurementTypeId to match other tables' columns
MeasurementType.rename(columns={"Id": "MeasurementTypeId"}, inplace=True)

# Overview
MeasurementType

,MeasurementTypeId,Measurement
1,2,EQ
2,3,SomaticCellCount
3,4,Fat
4,5,Protein
5,6,Bacteria
6,7,Urea
7,8,ES
10,11,FreezingPoint


In [15]:
# A measurement value at 0 isn't admissible in the recommandation process, which data is concerned?
measurement_types = list(MeasurementType["Measurement"])
(MilkMeasurements[measurement_types] == 0).sum(
    axis=1
).value_counts().sort_index().reset_index().rename(
    columns={"index": "NumberOfZeros", 0: "Count"}
)

,NumberOfZeros,Count
0,0,2293
1,1,697
2,2,158
3,3,83
4,4,2
5,7,72
6,8,45


In [16]:
# Keep rows only if NumberOfZeros <= 2
MilkMeasurements = MilkMeasurements.loc[
    (MilkMeasurements[measurement_types] == 0).sum(axis=1) <= 2
]

In [17]:
# Melt MilkMeasurements to pivot MilkMeasurement columns into rows
id_vars = ["Id", "ManagementAreaId"]
MeltedMilkMeasurements = MilkMeasurements.melt(
    id_vars=id_vars,
    var_name="Measurement",
    value_vars=measurement_types,
    value_name="Value",
)

# Merging with MeasurementType to get MeasurementTypeId
MeltedMilkMeasurements = pd.merge(
    MeltedMilkMeasurements, MeasurementType, on="Measurement"
)

# Overview
MeltedMilkMeasurements.head(10)

,Id,ManagementAreaId,Measurement,Value,MeasurementTypeId
0,1485,750,EQ,7.75,2
1,1486,750,EQ,7.93,2
2,1492,750,EQ,7.99,2
3,1493,750,EQ,7.78,2
4,1499,750,EQ,7.10,2
5,1500,750,EQ,7.74,2
6,1506,750,EQ,7.92,2
7,1522,750,EQ,8.03,2
8,1523,750,EQ,7.75,2
9,1529,750,EQ,7.95,2


# Measurement ranges

In [18]:
# A reference table to determine whether a collected sample needs an intervention (botanic pill) or not depending on the farm
MeasurementRange = pd.read_excel(original / "MeasurementRange.xlsx")
MeasurementRange.head()

,Id,ManagementAreaId,TargetValue,DangerZoneLow,LowValue,HighValue,DangerZoneHigh,Rank,MeasurementTypeId
0,1029,750,1.5,0.52,1.20,1.80,2.48,10,1
1,1031,1917,200.0,40.00,100.00,250.00,600.00,10,3
2,949,1898,850.0,170.00,425.00,1062.50,2550.00,8,3
3,950,1898,4.9,2.94,4.41,5.39,6.86,7,4
4,1032,1917,250.0,200.00,225.00,275.00,300.00,9,7


In [19]:
# Divide boundary values by 1000 for FreezingPoint
columns_to_divide = [
    "TargetValue",
    "DangerZoneLow",
    "LowValue",
    "HighValue",
    "DangerZoneHigh",
]
MeasurementRange.loc[MeasurementRange["MeasurementTypeId"] == 11, columns_to_divide] = (
    MeasurementRange.loc[MeasurementRange["MeasurementTypeId"] == 11, columns_to_divide]
    / 1000
)

# Merge measurement ranges with (melted) milk measurements and assign the corresponding trigger value

In [20]:
# Same number of Ids
print(f"MilkMeasurements: {len(MilkMeasurements)} rows")
print(
    f"MeltedMilkMeasurements (initial state): {len(MeltedMilkMeasurements['Id'].unique())} unique ids"
)

# Merging measurements with ranges for each farm -> some records are lost because of no matches in MeasurementRange
MeltedMilkMeasurements = pd.merge(
    MeltedMilkMeasurements,
    MeasurementRange.drop(columns="Id"),
    on=["ManagementAreaId", "MeasurementTypeId"],
)
print(
    f"MeltedMilkMeasurements (after merging): {len(MeltedMilkMeasurements['Id'].unique())} unique ids"
)

# Parameters with value = 0 are not taken into account in the recommandation system -> no record lost
MeltedMilkMeasurements = MeltedMilkMeasurements[MeltedMilkMeasurements["Value"] != 0]
print(
    f"MeltedMilkMeasurements (after filtering): {len(MeltedMilkMeasurements['Id'].unique())} unique ids"
)

MilkMeasurements: 3148 rows
MeltedMilkMeasurements (initial state): 3148 unique ids
MeltedMilkMeasurements (after merging): 2761 unique ids
MeltedMilkMeasurements (after filtering): 2761 unique ids


In [21]:
# Assign the corresponding trigger value
# - Adequate: nothing to do, everythoing is okay
# - Low / High: recommend a botanic pill (BP)
# - Danger Zone Low / High: notification -> a human needs to look more carefully
def assign_TriggerValue(Value, DangerZoneLow, LowValue, HighValue, DangerZoneHigh):
    if Value < DangerZoneLow:
        return "Danger Zone Low"
    elif Value < LowValue:
        return "Low"
    elif Value <= HighValue:
        return "Adequate"
    elif Value < DangerZoneHigh:
        return "High"
    elif Value >= DangerZoneHigh:
        return "Danger Zone High"


MeltedMilkMeasurements["TriggerValue"] = MeltedMilkMeasurements.apply(
    lambda row: assign_TriggerValue(
        row["Value"],
        row["DangerZoneLow"],
        row["LowValue"],
        row["HighValue"],
        row["DangerZoneHigh"],
    ),
    axis=1,
)

In [22]:
# Select useful columns and sort the dataset
MeltedMilkMeasurements = MeltedMilkMeasurements[
    [
        "Id",
        "ManagementAreaId",
        "MeasurementTypeId",
        "Measurement",
        "Value",
        "Rank",
        "TriggerValue",
    ]
]
MeltedMilkMeasurements = MeltedMilkMeasurements.sort_values(
    ["Id", "MeasurementTypeId"], ignore_index=True
)
MeltedMilkMeasurements.head()

,Id,ManagementAreaId,MeasurementTypeId,Measurement,Value,Rank,TriggerValue
0,1401,750,3,SomaticCellCount,1273.00,9,High
1,1401,750,4,Fat,4.47,7,Low
2,1401,750,8,ES,8.28,8,Low
3,1402,750,3,SomaticCellCount,1288.00,9,High
4,1402,750,4,Fat,4.66,7,Low


# Export

In [23]:
MeltedMilkMeasurements.to_csv(output / "MeltedMilkMeasurements.csv", index=False)

# Data labelling

In [24]:
# Based on the set of trigger values related to a milk measurement, we are going to assign different labels to our data
MilkMeasurements["Target"] = None

# Data to assign labels to
MeasurementIds = MeltedMilkMeasurements["Id"].unique()
MilkMeasurements = MilkMeasurements[MilkMeasurements["Id"].isin(MeasurementIds)]

# Set of trigger values per id
TriggerValuesSet = MeltedMilkMeasurements.groupby("Id").agg({"TriggerValue": set})
print(TriggerValuesSet.sample(5))

                           TriggerValue
Id                                     
2996              {High, Low, Adequate}
4616           {High, Danger Zone High}
5021                   {High, Adequate}
3413  {Low, Adequate, Danger Zone High}
3962      {High, Low, Danger Zone High}


## 1 No treatment if all params are OK

In [25]:
NoTreatmentIds = TriggerValuesSet[
    TriggerValuesSet["TriggerValue"] == {"Adequate"}
].index
MilkMeasurements.loc[MilkMeasurements["Id"].isin(NoTreatmentIds), "Target"] = (
    "No treatment"
)

## 2 Veterinarian intervention if at least one param is in danger zone

In [26]:
NotifIds = TriggerValuesSet[
    TriggerValuesSet["TriggerValue"].apply(
        lambda TriggerValues: sum(
            ["Danger" in TriggerValue for TriggerValue in TriggerValues]
        )
        > 0
    )
].index
MilkMeasurements.loc[MilkMeasurements["Id"].isin(NotifIds), "Target"] = (
    "Veterinarian intervention"
)

## 3 Find the right BP if no param in danger zone and at least one inadequate param

In [27]:
# Output 1: botanic pills are not detailed
MilkMeasurements.loc[MilkMeasurements["Target"].isna(), "Target"] = "Treatment"
MilkMeasurements.to_csv(output / "MilkMeasurementsPlusGroupedTarget.csv", index=False)

In [28]:
# Output 2: botanic pills are detailed
MilkMeasurements.loc[MilkMeasurements["Target"] == "Treatment", "Target"] = None

### Supplement matrix value: Botanic pill (BP) recommandations

In [29]:
# Load BP recommandations table
SupplementMatrixValue = pd.read_excel(original / "SupplementMatrixValue.xlsx")

In [30]:
# Inadequate values Ids
InadequateIds = {*MeasurementIds}.difference(list(NoTreatmentIds) + list(NotifIds))

# All botanic pills for each inadequate param
BotanicPillsList = (
    MeltedMilkMeasurements[MeltedMilkMeasurements["Id"].isin(InadequateIds)]
    .drop(columns=["Value", "Rank"])
    .merge(
        SupplementMatrixValue.drop(columns="Id"),
        on=["MeasurementTypeId", "TriggerValue"],
    )
    .sort_values(["Id", "MeasurementTypeId", "SupplementId"], ignore_index=True)
)

# Sum of ranks per sample or measurement
BotanicPillsSummedList = (
    BotanicPillsList.groupby(["Id", "SupplementId"])["Rank"]
    .sum()
    .reset_index()
    .rename(columns={"Rank": "RankSum"})
)

# Maximum sum of ranks (it is possible to have many BPs for one sample in case of multiple maximums)
BotanicPillsRank = BotanicPillsSummedList.groupby("Id")["RankSum"].max().reset_index()

In [31]:
# Recommended BP (or list of)
BotanicPills = BotanicPillsSummedList.merge(
    BotanicPillsRank, on=["Id", "RankSum"]
).rename(columns={"SupplementId": "Target"})
BotanicPills.head()

,Id,Target,RankSum
0,1401,1,18
1,1402,1,18
2,1403,1,18
3,1404,1,18
4,1405,1,18


In [32]:
# Is there a unique recommendation or not?
BotanicPillsCount = BotanicPills.groupby("Id").size()

In [33]:
# Assign the selected BP when it's unique
UniqueBPIds = BotanicPillsCount[BotanicPillsCount == 1].index

# Merge data, update subset values, and drop unnecessary columns
MilkMeasurements = MilkMeasurements.merge(
    BotanicPills.loc[BotanicPills["Id"].isin(UniqueBPIds), ["Id", "Target"]],
    on="Id",
    how="left",
    suffixes=("_original", "_updated"),
)
MilkMeasurements["Target"] = MilkMeasurements["Target_updated"].fillna(
    MilkMeasurements["Target_original"]
)
MilkMeasurements.drop(columns=["Target_original", "Target_updated"], inplace=True)

In [34]:
# Else, ... There are many maximums when adding up the ranks
ManyBPIds = BotanicPillsCount[BotanicPillsCount >= 2].index

In [35]:
# We must find the number of params included in the sum
NumberOfParams = (
    BotanicPillsList.loc[BotanicPillsList["Id"].isin(ManyBPIds)]
    .rename(columns={"SupplementId": "Target"})
    .merge(BotanicPills, on=["Id", "Target"])
    .groupby(["Id", "Target"])
    .size()
    .reset_index()
    .rename(columns={0: "NumberOfParams"})
)

# The choosen BP is the one that cures the most params (there may be several maximums with the same number of params)
MaxNumberOfParams = NumberOfParams.groupby("Id")["NumberOfParams"].max().reset_index()
MaxNumberOfParams = NumberOfParams.merge(MaxNumberOfParams, on=["Id", "NumberOfParams"])

# Number of distinct BPs
DistinctMaxNumberOfParams = MaxNumberOfParams.groupby("Id").size()

In [36]:
# If the previous table contains only one record per id, then there's a BP that curates strictly more params than the others; this is the one chosen
UniqueMaxIds = DistinctMaxNumberOfParams[
    DistinctMaxNumberOfParams == 1
].index.sort_values()

# Merge data, update subset values, and drop unnecessary columns
MilkMeasurements = MilkMeasurements.merge(
    MaxNumberOfParams.loc[MaxNumberOfParams["Id"].isin(UniqueMaxIds), ["Id", "Target"]],
    on="Id",
    how="left",
    suffixes=("_original", "_updated"),
)
MilkMeasurements["Target"] = MilkMeasurements["Target_updated"].fillna(
    MilkMeasurements["Target_original"]
)
MilkMeasurements.drop(columns=["Target_original", "Target_updated"], inplace=True)

In [37]:
# If there are many top BPs, add all
ManyMaxIds = DistinctMaxNumberOfParams[
    DistinctMaxNumberOfParams > 1
].index.sort_values()

# Merge data, update subset values, and drop unnecessary columns
MilkMeasurements = MilkMeasurements.merge(
    BotanicPills.loc[BotanicPills["Id"].isin(ManyMaxIds), ["Id", "Target"]]
    .groupby("Id")["Target"]
    .apply(list)
    .reset_index(),
    on="Id",
    how="left",
    suffixes=("_original", "_updated"),
)
MilkMeasurements["Target"] = MilkMeasurements["Target_updated"].fillna(
    MilkMeasurements["Target_original"]
)
MilkMeasurements.drop(columns=["Target_original", "Target_updated"], inplace=True)

# Export final dataset (with target)

In [38]:
# Columns order
MilkMeasurements = MilkMeasurements[
    [
        "Id",
        "ManagementAreaId",
        "SampleDate",
        "EQ",
        "SomaticCellCount",
        "Fat",
        "Protein",
        "Bacteria",
        "Urea",
        "ES",
        "FreezingPoint",
        "Breed",
        "Species",
        "Target",
    ]
]

In [39]:
MilkMeasurements.to_csv(output / "MilkMeasurementsPlusDetailedTarget.csv", index=False)